# Last API with pylast

[Oficial Website](https://www.last.fm/api/)

[PyLast Repo](https://github.com/pylast)

In [2]:
import pandas as pd 
import numpy as np 
import pylast

import os
import json
import sys
import requests

sys.path.append('../scripts/') 

from create_last_database import User

In [3]:
API_KEY = input()
API_SECRET = input()

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)